# 🌿 Extended Analysis: Green Inference Scheduling

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load all necessary CSVs
scheduled_df = pd.read_csv('../results/execution_schedule.csv', parse_dates=['original_time', 'scheduled_time'])
jobs_df = pd.read_csv('../data/inference_jobs.csv', parse_dates=['timestamp'])
carbon_df = pd.read_csv('../data/carbon_intensity.csv', parse_dates=['timestamp'])

## 1. CO₂ Emissions: Baseline vs Scheduled

In [ ]:
baseline_df = pd.merge(jobs_df, carbon_df, left_on='timestamp', right_on='timestamp', how='left')
baseline_df['carbon_emitted'] = baseline_df['carbon_intensity'] * baseline_df['expected_power_kwh'] / 1000
scheduled_df['carbon_emitted'] = scheduled_df['carbon_intensity'] * scheduled_df['power_kwh'] / 1000

emissions = {
    'Baseline': baseline_df['carbon_emitted'].sum(),
    'Scheduled': scheduled_df['carbon_emitted'].sum()
}
plt.figure(figsize=(6, 4))
sns.barplot(x=list(emissions.keys()), y=list(emissions.values()), palette='Set2')
plt.title('CO₂ Emissions: Baseline vs Scheduled')
plt.ylabel('kg CO₂')
plt.show()

## 2. Job Execution Time Shift

In [ ]:
scheduled_df['delay_hours'] = (scheduled_df['scheduled_time'] - scheduled_df['original_time']).dt.total_seconds() / 3600
plt.figure(figsize=(8, 4))
sns.histplot(scheduled_df['delay_hours'], bins=10)
plt.title('Job Execution Delay (Hours)')
plt.xlabel('Delay (hours)')
plt.ylabel('Number of Jobs')
plt.show()

## 3. Solar Power Used Per Hour

In [ ]:
solar_usage = scheduled_df.groupby(scheduled_df['scheduled_time'].dt.hour)['solar_kw_used'].sum()
plt.figure(figsize=(10, 4))
solar_usage.plot(kind='bar', color='green')
plt.title('Solar Power Used Per Hour (Scheduled Jobs)')
plt.xlabel('Hour of Day')
plt.ylabel('Solar kW Used')
plt.show()

## 4. Job Timeline by Carbon Intensity and Energy Source

In [ ]:
color_map = {'solar': 'green', 'grid': 'gray'}
plt.figure(figsize=(12, 5))
sns.scatterplot(data=scheduled_df, x='scheduled_time', y='carbon_intensity', hue='source', palette=color_map)
plt.title('Job Timeline: Carbon Intensity vs Time by Energy Source')
plt.xlabel('Scheduled Time')
plt.ylabel('Carbon Intensity (gCO₂/kWh)')
plt.legend(title='Energy Source')
plt.show()